[Adaptado do repositório ["Digital-Twin-in-python"](https://github.com/Javihaus/Digital-Twin-in-python), conforme descrito no texto ["How to Build a Digital Twin in Python"](https://medium.com/towards-data-science/how-to-build-a-digital-twin-b31058fd5d3e)]

![image](https://github.com/fabiobento/dnn-course-2025-1/raw/main/nn-intro/images/digital_twin.png)

# Gêmeos Digitais em Python

- Neste caderno, mostraremos como criar um gêmeo digital simples, mas útil, usando python.
- Nosso ativo será uma bateria de íons de lítio.
- Esse gêmeo digital nos permitirá modelar e prever o comportamento das baterias e pode ser incluído em qualquer processo de gerenciamento de ativos virtuais.

# Gêmeo digital híbrido de uma bateria de íons de lítio


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

### 1. Carregar dados experimentais

In [7]:
df = pd.read_csv('nn-intro/discharge.csv')

In [8]:
df = df[df['Battery'] == 'B0005']
df = df[df['Temperature_measured'] > 36] #escolher a bateria B0005
#df['Time'] =pd.to_datetime(df['Time'], unit='s')
dfb = df.groupby(['id_cycle']).max()
dfb['Cumulated_T'] = dfb['Time'].cumsum()

In [9]:
import plotly.express as px
fig = px.scatter_matrix(dfb.drop(columns=['Time','type', 'ambient_temperature', 
                                          'time', 'Battery']), 
                                )
fig.update_traces(marker=dict(size=2,
                              color='crimson',
                              symbol='square')),
fig.update_traces(diagonal_visible=False)
fig.update_layout(
    title='Conjunto de dados da Bateria',
    width=900,
    height=1200,
)
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white',
                    font=dict(size=7)
                    )

fig.show()

In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb['Cumulated_T']/3600, 
                         y=dfb['Capacity'],
                         mode='lines',
                         name='Capacity',
                         marker_size=3, 
                         line=dict(color='crimson', width=3)    
                        ))
fig.update_layout(
                  title="Capacidade de descarga da bateria",
                  xaxis_title="Tempo de trabalho [horas]",
                  yaxis_title=f"Capacidade da bateria em Ahr"
    )
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white')

### 2. Definir um modelo físico

Modelo físico de acordo com [[1](https://ieeexplore.ieee.org/abstract/document/7488267?casa_token=k6OpK4NKQcQAAAAA:BMPFW5WsPTb5ERn8Z_lMkBrxkWXYwW6QgDwAMUj3jQy44YNui8c87Qbgv3PfvEsdZh9imwMWtMbJ)]. A equação básica é:$$L = 1 - (1 - L' )e^{-f_d}$$

Onde $L$ é a vida útil da bateria e $L'$ é a vida útil inicial da bateria.

$f_d$ é uma taxa de degradação linearizada por unidade de tempo e por ciclo.  Ela pode ser descrita como: $$f_d = f_d(t, δ, σ, T_c)$$ em que $t$ é o tempo de carregamento, δ é a profundidade de descarga do ciclo, σ é o estado médio de carga do ciclo e $T_c$ é a temperatura da célula.

A equação da capacidade de bateria pode ser escrita da seguinte forma: $$C = C_0e^{f_d}$$

Descobriu-se empiricamente que $f_d$ se aproxima de: $$f_d = \frac{kT_Ci}{t}$$
onde $k= $ 0,13, $i$ o número do ciclo e $t$ o tempo de carga para cada ciclo.

In [11]:
from math import e
L = (dfb['Capacity']-dfb['Capacity'].iloc[0:1].values[0])/-dfb['Capacity'].iloc[0:1].values[0]
K = 0.13
L_1 = 1-e**(-K*dfb.index*dfb['Temperature_measured']/(dfb['Time']))
dfb['C. Capacity'] = -(L_1*dfb['Capacity'].iloc[0:1].values[0]) + dfb['Capacity'].iloc[0:1].values[0]

In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['C. Capacity'],
                         mode='lines',
                         name='Physical model',
                         line=dict(color='navy', 
                                   width=2.5,
                                   )))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name='NASA dataset',
                         line_color='navy'))
fig.update_layout(
    title="Comparação do modelo físico com o dataset",
    xaxis_title="Ciclos",
    yaxis_title="𝐶, Capacidade [Ahr]")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.8
))

fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

### 3. Comparar os dados experimentais com o modelo físico

In [24]:
# Erro médio absotulo
M = pd.DataFrame()
S = pd.DataFrame()
def MAE(M,S):    
    return np.sum(S-M)/len(S)

print(f'Erro médio absoluto =', round(MAE(dfb['Capacity'], dfb['C. Capacity']), 3))

Erro médio absoluto = 0.004


### 4. Gêmeo digital híbrido 

In [ ]:
# Definir os dados de entrada e saída
X_in = dfb['C. Capacity']          # entrada:  input: a série temporal da simulação
X_out = dfb['Capacity'] - dfb['C. Capacity']   # saída: diferença entre medição e simulação
X_in_train, X_in_test, X_out_train, X_out_test = train_test_split(X_in, X_out, test_size=0.33)

In [15]:
X_in_train.shape

(112,)

In [ ]:
#A função Dense no Keras constrói uma camada de rede neural totalmente conectada, inicializando automaticamente tanto os pesos como as polarizações.
#Primeira camada oculta
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [ ]:
epochs = 100
loss = "mse"
model.compile(optimizer='adam',
              loss=loss,
              metrics=['mae'], #Erro médio absoluto
             )
history = model.fit(X_in_train, X_out_train, 
                    shuffle=True, 
                    epochs=epochs,
                    batch_size=20,
                    validation_data=(X_in_test, X_out_test), 
                    verbose=1)

2025-02-25 09:44:55.875017: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Epoch 1/100


2025-02-25 09:44:56.000833: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-25 09:44:56.008552: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-25 09:44:56.015285: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1/6 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 0.0079 - mae: 0.0813

I0000 00:00:1740487497.402121   20103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - loss: 0.0043 - mae: 0.0532 - val_loss: 0.0023 - val_mae: 0.0422
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0014 - mae: 0.0309 - val_loss: 0.0013 - val_mae: 0.0262
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0015 - mae: 0.0313 - val_loss: 0.0013 - val_mae: 0.0328
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0012 - mae: 0.0297 - val_loss: 9.1765e-04 - val_mae: 0.0255
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 9.2066e-04 - mae: 0.0244 - val_loss: 9.0257e-04 - val_mae: 0.0249
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 9.6348e-04 - mae: 0.0259 - val_loss: 9.8727e-04 - val_mae: 0.0277
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.5073e-04 - mae: 0.0246 - val_loss: 8.8902e-04 - val_mae: 0.0245
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.8930e-04 - mae: 0.0241 - val_loss: 0.0010 - val_mae: 0.0280
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step -

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(0, epochs, 1),
                         y=history.history['mae'],
                         mode='lines',
                         name=f'MAE de Treino',
                         marker_size=3, 
                         line_color='orange'))
fig.add_trace(go.Scatter(x=np.arange(0, epochs, 1),
                         y=history.history['val_mae'],
                         mode='lines',
                         name=f'MAE de Validação',
                         line_color='grey'))

fig.update_layout(
                  title="Treinamento da rede",
                  xaxis_title="Épocas",
                  yaxis_title=f"Erro médio absoluto")
fig.update_layout({'plot_bgcolor': '#f2f8fd' , 
                   'paper_bgcolor': 'white',}, 
                   template='plotly_white')

### 4. Compile the hybrid digital twin

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_in_train, 
                         y=X_out_train,
                         mode='markers',
                         name=f'Modelled Capacity',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ), 
                        line_color='crimson'))
fig.add_trace(go.Scatter(x = X_in_train, 
                         y=model.predict(X_in_train).reshape(-1),
                         mode='lines',
                         name=f'Trained Capacity',
                         line=dict(color='navy', width=3)))
fig.update_layout(
    title="Network training",
    xaxis_title="Modelled capacity",
    yaxis_title="Δ (Mod. Capacity - Measured Cap.)")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.45
))
fig.update_layout({'plot_bgcolor': '#f2f8fd' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [20]:
X_twin = X_in + model.predict(X_in).reshape(-1)

fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=X_twin,
                         mode='lines',
                         name=f'Hybrid digial twin',
                         line=dict(color='firebrick', width=3)))
fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['C. Capacity'],
                         mode='lines',
                         name=f'Modelled capacity',
                         line=dict(color='navy', 
                                   width=3,
                                   dash='dash')))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name=f'Observed capacity',
                         line_color='navy'))
fig.update_layout(
    title="Comparison of hybrid twin with other models",
    xaxis_title="Cycles",
    yaxis_title="Capacity in Ahr")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.77
))
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


## 5. Prediction whit hybrid twin model


In [21]:
cycles = np.arange(168,500,1)
temperature = dfb['Temperature_measured'].iloc[167]
time = dfb['Time'].iloc[167]
K = 0.13
L_e = 1-e**(-K*cycles*temperature/time)
X_in_e = -(L_e*dfb['Capacity'].iloc[0:1].values[0]) + dfb['Capacity'].iloc[0:1].values[0]
C_twin_e = X_in_e + model.predict(X_in_e).reshape(-1)

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [ ]:
X_twin = X_in + model.predict(X_in).reshape(-1)

fig = go.Figure()

fig.add_trace(go.Scatter(x=cycles, 
                         y=X_in_e,
                         mode='lines',
                         name=f'C modelled (predicted)',
                         line=dict(color='navy', 
                                   width=3,
                                   dash='dash')))
fig.add_trace(go.Scatter(x=cycles, 
                         y=C_twin_e,
                         mode='lines',
                         name=f'C Digital twin (predicted)',
                         line=dict(color='crimson', 
                                   width=3,
                                   dash='dash'
                                  )))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=X_twin,
                         mode='lines',
                         name=f'C Digital twin',
                         line=dict(color='crimson',
                                   width=2)))
fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['C. Capacity'],
                         mode='lines',
                         name=f'C modelled',
                         line=dict(color='navy', 
                                   width=2)))

fig.update_layout(
    title="Battery capacity prediction",
    xaxis_title="Cycles",
    yaxis_title="Battery capacity [Ahr]")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.72
))
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

REFERÊNCIAS

[1] [XU, Bolun et al. Modeling of lithium-ion battery degradation for cell life assessment. IEEE Transactions on Smart Grid, v. 9, n. 2, p. 1131-1140, 2016.](https://ieeexplore.ieee.org/abstract/document/7488267?casa_token=k6OpK4NKQcQAAAAA:BMPFW5WsPTb5ERn8Z_lMkBrxkWXYwW6QgDwAMUj3jQy44YNui8c87Qbgv3PfvEsdZh9imwMWtMbJ)

[2] [LARESGOITI, Izaro et al. Modeling mechanical degradation in lithium ion batteries during cycling: Solid electrolyte interphase fracture. Journal of Power Sources, v. 300, p. 112-122, 2015.](https://www.sciencedirect.com/science/article/pii/S0378775315302949?casa_token=2ROi2uYsB_YAAAAA:d6b_VcLQFD9uleLgbIK7ENnVVzS23qKCEwdeFqRA5nVKA86WHhucSBlPJXO7mGqYPiO8Nq0L-Q)
